In [1]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import cv2

def create_temporal_rgb_frame(df, ts_0, ts_1, height=200, width=200):
    """
    Create a temporal RGB frame from event data within a given time window.

    Args:
        df (pd.DataFrame): DataFrame containing event data with columns ['t', 'x', 'y', 'p'].
        ts_0 (float): Start timestamp (in microseconds).
        ts_1 (float): End timestamp (in microseconds).
        height (int): Height of the output image.
        width (int): Width of the output image.

    Returns:
        np.ndarray: An (height, width, 3) uint8 RGB image where each color channel encodes events
                    from a different third of the time window:
                        - Red: events in the first third,
                        - Green: events in the middle third,
                        - Blue: events in the last third.
    """
    ev_frame = np.zeros([height, width, 3], dtype=np.uint8)
    
    if df.empty or (ts_1 <= ts_0):
        return ev_frame

    window = ts_1 - ts_0
    w1 = ts_0 + window / 3
    w2 = ts_1 - window / 3

    # --- red (1/3) ---
    red_ev = df.loc[df['t'].between(ts_0, w1)]
    if not red_ev.empty:
        x = red_ev['x'].to_numpy()
        y = red_ev['y'].to_numpy()
        t = red_ev['t'].to_numpy()
        
        alpha = (t - ts_0) / window
        ev_frame[y, x, 0] = np.clip(alpha * 255.0, 0, 255)

    # --- green (2/3) ---
    green_ev = df.loc[df['t'].between(w1, w2)]
    if not green_ev.empty:
        x = green_ev['x'].to_numpy()
        y = green_ev['y'].to_numpy()
        t = green_ev['t'].to_numpy()

        alpha = (t - ts_0) / window
        ev_frame[y, x, 1] = np.clip(alpha * 255.0, 0, 255)

    # --- blue (3/3) ---
    blue_ev = df.loc[df['t'].between(w2, ts_1)]
    if not blue_ev.empty:
        x = blue_ev['x'].to_numpy()
        y = blue_ev['y'].to_numpy()
        t = blue_ev['t'].to_numpy()

        alpha = (t - ts_0) / window
        ev_frame[y, x, 2] = np.clip(alpha * 255.0, 0, 255)
        
    return ev_frame

def generate_event_images(filename, domain='train' , outdir='./dataset/train', divider=1, img_height=200, img_width=200):
    """
    Generate and save temporal event images from event-based data.

    Args:
        filename (str): Name of the .npz file (without extension) containing event data.
        domain (str): Subdirectory under './data' and './dataset' (e.g., 'train', 'test').
        outdir (str): Output directory to save generated images.
        divider (int): Number of subdivisions per timestamp interval for interpolation.
        img_height (int): Height of the output images.
        img_width (int): Width of the output images.
    """
    datapath = './data'
    fpath = os.path.join(datapath, domain, filename + '.npz')
    outpath = os.path.join(outdir, filename + '_' + str(divider), 'img')
    os.makedirs(outpath, exist_ok=True)
    
    sequence = np.load(fpath)
    events = sequence['events']
    timestamps = sequence['timestamps'] 
    # traj = sequence['traj']
    # range_meter = sequence['range_meter']

    ev_data = pd.DataFrame(events, columns=['t', 'x', 'y', 'p']).set_index('t').sort_index()
    # ts_data = pd.DataFrame(t, columns=['time/s'])
    # traj_data = pd.DataFrame(x, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'roll', 'pitch', 'yaw','wr', 'wp', 'wy'])
    # rm_data = pd.DataFrame(rm, columns=['time/s', 'd/m'])

    new_ts_list = []
    if divider == 1:
        new_ts_list = timestamps
        ts_us = new_ts_list * 1e6
    else: 
        for i in range(len(timestamps) - 1):
            start_time = timestamps[i]
            end_time = timestamps[i+1]
            interpolated_times = np.linspace(start_time, end_time, divider, endpoint=False)
            new_ts_list.append(interpolated_times)
    
        new_ts_list.append(np.array([timestamps[-1]]))
        ts_us = np.concatenate(new_ts_list) * 1e6
        
    event_groups_list = []

    for i in tqdm(range(len(ts_us))):
        if (i != len(ts_us) - 1): 
            ts_0 = ts_us[i]
            ts_1 = ts_us[i+1]
            df = ev_data.loc[ts_0 : ts_1 - 1].reset_index()
        else:
            ts_0 = ts_us[i]
            ts_1 = 2 * ts_0 - ts_us[i - 1]
            df = ev_data.loc[ts_0 : ts_1 - 1].reset_index()

        df_pos = df[df['p'] == True] 
        df_neg = df[df['p'] == False]
        
        rgb_pos = create_temporal_rgb_frame(df_pos, ts_0, ts_1, img_height, img_width)
        bgr_pos = cv2.cvtColor(rgb_pos, cv2.COLOR_RGB2BGR)
        imgname_pos = os.path.join(outpath, f"{i:04d}_pos.png")
        cv2.imwrite(imgname_pos, bgr_pos)

        rgb_neg = create_temporal_rgb_frame(df_neg, ts_0, ts_1, img_height, img_width)
        bgr_neg = cv2.cvtColor(rgb_neg, cv2.COLOR_RGB2BGR)
        imgname_neg = os.path.join(outpath, f"{i:04d}_neg.png")
        cv2.imwrite(imgname_neg, bgr_neg)

    print(f"Save {len(ts_us)} couples of imgs(pos/neg) at {outpath}. ")

def generate_traj_data(filename, domain='train' , outdir='./dataset/train', divider=1):
    """
    Generate and save interpolated trajectory data from a .npz file.

    Args:
        filename (str): Name of the .npz file (without extension) containing trajectory data.
        domain (str): Subdirectory under './data' and './dataset' (e.g., 'train', 'test').
        outdir (str): Output directory to save the generated CSV file.
        divider (int): Number of subdivisions per trajectory interval for interpolation.
    """
    datapath = './data'
    fpath = os.path.join(datapath, domain, filename + '.npz')
    outpath = os.path.join(outdir, filename + '_' + str(divider), 'data')
    os.makedirs(outpath, exist_ok=True)
    
    sequence = np.load(fpath)
    # events = sequence['events']
    # timestamps = sequence['timestamps'] 
    traj = sequence['traj']
    # range_meter = sequence['range_meter']

    # ev_data = pd.DataFrame(events)
    # ts_data = pd.DataFrame(t, columns=['time/s'])
    # traj_data = pd.DataFrame(x, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'roll', 'pitch', 'yaw','wr', 'wp', 'wy'])
    # rm_data = pd.DataFrame(rm, columns=['time/s', 'd/m'])

    traj_new_list = []

    if divider == 1:
        traj_new = traj
    else: 
        for i in range(len(traj) - 1):
            start_vec = traj[i] 
            end_vec = traj[i+1] 
            interpolated_vec = np.linspace(start_vec, end_vec, divider, endpoint=False)
            traj_new_list.append(interpolated_vec)

        traj_new_list.append(traj[-1].reshape(1, -1))
        traj_new = np.concatenate(traj_new_list, axis=0)
    
    df = pd.DataFrame(traj_new, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'roll', 'pitch', 'yaw','wr', 'wp', 'wy'])
    csvpath = os.path.join(outpath, 'trajectory.csv')
    df.to_csv(csvpath, index=False)
    print(f"Save {len(df)} lines of trajectory data to {csvpath}.")

In [24]:
for i in [1, 2, 4]:
    for j in range(0, 28):
        file_num = str(j).zfill(4)
        generate_event_images(file_num, divider=i)
        generate_traj_data(file_num, divider=i)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:03<00:00, 32.26it/s]


Save 120 couples of imgs(pos/neg) at ./dataset/train/0000_1/img. 
Save 120 lines of trajectory data to ./dataset/train/0000_1/data/trajectory.csv.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:02<00:00, 43.26it/s]


Save 120 couples of imgs(pos/neg) at ./dataset/train/0001_1/img. 
Save 120 lines of trajectory data to ./dataset/train/0001_1/data/trajectory.csv.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:01<00:00, 105.69it/s]


...

Save 477 couples of imgs(pos/neg) at ./dataset/train/0025_4/img. 
Save 477 lines of trajectory data to ./dataset/train/0025_4/data/trajectory.csv.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 477/477 [00:02<00:00, 206.09it/s]


Save 477 couples of imgs(pos/neg) at ./dataset/train/0026_4/img. 
Save 477 lines of trajectory data to ./dataset/train/0026_4/data/trajectory.csv.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 477/477 [00:06<00:00, 78.65it/s]

Save 477 couples of imgs(pos/neg) at ./dataset/train/0027_4/img. 
Save 477 lines of trajectory data to ./dataset/train/0027_4/data/trajectory.csv.


In [2]:
for i in [1, 2, 4]:
    for j in range(28, 93):
        file_num = str(j).zfill(4)
        generate_event_images(file_num, domain="test", divider=i)
        # generate_traj_data(file_num, divider=i)

100%|██████████| 120/120 [00:00<00:00, 198.54it/s]


Save 120 couples of imgs(pos/neg) at ./dataset/train/0028_1/img. 


100%|██████████| 120/120 [00:00<00:00, 224.39it/s]


Save 120 couples of imgs(pos/neg) at ./dataset/train/0029_1/img. 


100%|██████████| 120/120 [00:01<00:00, 99.71it/s] 


...

Save 477 couples of imgs(pos/neg) at ./dataset/train/0090_4/img. 


100%|██████████| 477/477 [00:10<00:00, 43.37it/s]


Save 477 couples of imgs(pos/neg) at ./dataset/train/0091_4/img. 


100%|██████████| 477/477 [00:12<00:00, 37.93it/s]

Save 477 couples of imgs(pos/neg) at ./dataset/train/0092_4/img. 
